# Structured Output Tutorial

## Overview

This tutorial delves into tehcnique of function calling in language models to achieve structured outputs from any LLM.

## Motivation

In the rapidly evolving field of artificial intelligence, the ability to generate structured outputs from language models is becoming increasingly important. Structured outputs allow for more precise and interpretable results, which are essential for applications ranging from data extraction to automated decision-making.

## Key Components

1. Understanding how to invoke specific functions within a language model to generate structured outputs.
2. Methods for ensuring that the generated outputs conform to the expected structure and format.
3. Practical examples of integrating structured output generation with popular frameworks like LangChain and OpenRouter.

## Setup

First, let's import the necessary libraries and set up our environment.

In [1]:
import json
import os
from datetime import datetime
from os import getenv

import requests
from dotenv import load_dotenv
from openai import OpenAI
from pydantic import BaseModel, Field

load_dotenv()

# Initialize the language model
llm = OpenAI(
    api_key=getenv("OPENROUTER_API_KEY"),
    base_url=getenv("OPENROUTER_BASE_URL"),
)

## Unstructured Output

In [2]:
# Define your prompt
prompt = "Can you tell me about Harry Potter's profile?"

# Make the LLM call
response = llm.chat.completions.create(
    model="azure/gpt-4o",
    messages=[{"role": "user", "content": prompt}],
)

# Print the result
print(response.choices[0].message.content)

Certainly! Harry Potter is the central character of the immensely popular "Harry Potter" series written by J.K. Rowling. Here's a detailed profile of Harry Potter:

### Basic Information
- **Full Name:** Harry James Potter
- **Date of Birth:** July 31, 1980
- **Place of Birth:** Godric's Hollow, England
- **Blood Status:** Half-blood (one magical parent, one non-magical parent)

### Physical Description
- **Hair:** Unruly black hair
- **Eyes:** Green
- **Scar:** Lightning-shaped scar on his forehead

### Family
- **Parents:** James Potter and Lily Potter (née Evans)
- **Godfather:** Sirius Black
- **Cousins:** Second cousins to the Weasley family through marriage (Lily being related to the Weasleys through her sister, Petunia Evans Dursley)
- **Spouse:** Ginny Weasley
- **Children:** James Sirius Potter, Albus Severus Potter, Lily Luna Potter

### Hogwarts House
- **House:** Gryffindor

### Wand
- **Wood:** Holly
- **Core:** Phoenix feather from Fawkes
- **Length:** 11 inches

### Key 

## Structured Output

![Structured Output](https://storage.googleapis.com/tqtensor-sharing/structured_output.png)

In [3]:
class Person(BaseModel):
    name: str = Field(description="The person's full name")
    date_of_birth: datetime = Field(
        description="The person's date of birth in ISO 8601 format (YYYY-MM-DD)"
    )
    occupation: str = Field(description="The person's current job or profession")


def get_person_info() -> Person:
    """Get information about a person from LLM response."""

    return Person(name="", age=0, occupation="")


# Create the function description for the API in tools format
tool_description = {
    "type": "function",
    "function": {
        "name": "get_person_info",
        "description": "Extract and structure information about a person from text. YOU MUST USE THIS FUNCTION to return the structured data about the person mentioned in the text. Do not respond with free-form text.",
        "parameters": Person.model_json_schema(),
        "required": ["name", "date_of_birth", "occupation"],
    },
}

# Setup headers for the LiteLLM proxy
headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer {}".format(getenv("OPENROUTER_API_KEY")),
}

# Make the LLM call via LiteLLM proxy
response = requests.post(
    "{}/v1/chat/completions".format(getenv("OPENROUTER_BASE_URL")),
    headers=headers,
    json={
        "model": "bedrock/nova-pro-v1",
        "messages": [{"role": "user", "content": prompt}],
        "tools": [tool_description],
        "tool_choice": "required",
    },
)

# Parse the response
response_data = response.json()

# Extract the tool call arguments
tool_calls = response_data["choices"][0]["message"].get("tool_calls", [])
if tool_calls:
    function_arguments = json.loads(tool_calls[0]["function"]["arguments"])

    # Create a Person object from the arguments
    person = Person(**function_arguments)

    # Print the result
    print(f"Name: {person.name}")
    print(f"DoB: {person.date_of_birth}")
    print(f"Occupation: {person.occupation}")
else:
    print("No tool calls found in the response")

Name: Harry Potter
DoB: 1980-07-31 00:00:00
Occupation: Wizard
